# Test Connection to HDFS and Load Data

This notebook tests the connection to HDFS and loads flood data for analysis.

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
import os

# Create Spark session
spark = SparkSession.builder \
    .appName("HDFS Connection Test") \
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true") \
    .getOrCreate()

# Test if Spark is working
print(f"Spark version: {spark.version}")
print(f"Spark master: {spark.sparkContext.master}")
print("Spark is working!")

In [ ]:
# First check for local file
try:
    # List files in the current directory
    print("Files in local directory:")
    !ls -la
    
    # Check if our data file exists
    !ls -la DataFixABD.csv
except Exception as e:
    print(f"Error checking local files: {e}")

In [ ]:
# Load local data first
try:
    # Try to read the local file
    df_local = spark.read.csv("DataFixABD.csv", header=True, inferSchema=True)
    
    # Show summary of the data
    print("Successfully read local CSV file")
    print(f"Number of records: {df_local.count()}")
    print("\nSchema:")
    df_local.printSchema()
    
    print("\nSample data:")
    df_local.show(5)
except Exception as e:
    print(f"Error reading local file: {e}")

In [ ]:
# Test connection to HDFS
try:
    # Use Hadoop FileSystem API to access HDFS
    conf = spark.sparkContext._jsc.hadoopConfiguration()
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(conf)
    
    # List contents of the root directory
    path = spark._jvm.org.apache.hadoop.fs.Path("/")
    if fs.exists(path):
        print("Successfully connected to HDFS!")
        status = fs.listStatus(path)
        
        print("\nContents of HDFS root directory:")
        for fileStatus in status:
            print(f"- {fileStatus.getPath().getName()}")
    else:
        print("HDFS root directory not found. Check if HDFS is running.")
except Exception as e:
    print(f"Error connecting to HDFS: {e}")

In [ ]:
# Now try to write local data to HDFS
try:
    # Create a sample dataframe if local data read failed
    if 'df_local' not in locals():
        from pyspark.sql.types import StructType, StructField, StringType, FloatType
        from datetime import datetime, timedelta
        
        # Create sample data
        data = [(datetime.now().strftime("%Y-%m-%d"), "Test Province", 1.0, 2.0, 3.0, 4.0)]
        schema = StructType([
            StructField("Tanggal", StringType(), True),
            StructField("Provinsi", StringType(), True),
            StructField("Nilai_Server", FloatType(), True),
            StructField("Nilai_Kartu", FloatType(), True),
            StructField("Nilai_Registered", FloatType(), True),
            StructField("Nilai_Unregistered", FloatType(), True)
        ])
        df_local = spark.createDataFrame(data, schema)
    
    # Write to HDFS
    print("Writing data to HDFS...")
    hdfs_path = "hdfs://namenode:8020/user/jovyan/flood_data"
    df_local.write.mode("overwrite").csv(hdfs_path, header=True)
    print(f"Successfully wrote data to HDFS at {hdfs_path}")
    
    # Verify the write by reading back
    print("\nVerifying by reading back from HDFS...")
    df_hdfs = spark.read.csv(hdfs_path, header=True, inferSchema=True)
    print(f"Number of records read back: {df_hdfs.count()}")
    df_hdfs.show(5)
    
except Exception as e:
    print(f"Error writing to HDFS: {e}")

In [ ]:
# Clean up Spark session
spark.stop()
print("Spark session stopped.")